# Testing and evaluation

**Test-train split**: training models on a portion of the original dataset (called **training data**) and then evaluating it on the remainder (called **testing data**)

You can use the Python library `scikit-learn` to perform the split:

```python
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,
    train_size=0.80,
    test_size=0.20,
    random_state=2) 
```

**Note**: `random_state = 2` sets a random state, so that we create the same train-test split each time we run our code

## Evaluation

Use the testing data to evaluate the model.
- `model.eval()` sets the model to evaluation mode
- `with torch.no_grad()` turns off gradient calculations (which we don’t need outside of training)

```python
model.eval()
with torch.no_grad():
    predictions = model(X_test)
    test_MSE = loss(predictions, y_test)
```

## Saving and loading models

```python
# save the neural network to a specified path
torch.save(model, 'model.pth')

# load the saved neural network from the specified path
loaded_model = torch.load('model.pth')
```

**Note**: `model.pth` is the file extension for PyTorch models

### Example

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

apartments_df = pd.read_csv("streeteasy.csv")

numerical_features = ['bedrooms', 'bathrooms', 'size_sqft', 'min_to_subway', 'floor', 'building_age_yrs',
                      'no_fee', 'has_roofdeck', 'has_washer_dryer', 'has_doorman', 'has_elevator', 'has_dishwasher',
                      'has_patio', 'has_gym']

# create tensor of input features
X = torch.tensor(apartments_df[numerical_features].values, dtype=torch.float)
# create tensor of targets
y = torch.tensor(apartments_df['rent'].values, dtype=torch.float).view(-1,1)

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, test_size=0.20, random_state=2)

# define the model using nn.Sequential
model = nn.Sequential(
    nn.Linear(14, 128),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.ReLU(),
    nn.Linear(64, 1)
)

# MSE loss function + optimizer
loss = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# training loop
num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    predictions = model(X_train)
    train_MSE = loss(predictions, y_train)
    train_MSE.backward()
    optimizer.step()

    # keep track of the loss during training
    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], MSE Loss: {train_MSE.item()}')

# save the model
torch.save(model, 'model.pth')

# load the model
loaded_model = torch.load('model.pth')

# evaluate the model
loaded_model.eval()
with torch.no_grad():
    predictions = loaded_model(X_test)
    test_MSE = loss(predictions, y_test)
    print(f'Test MSE: {test_MSE.item()}')
